In [65]:
import db_operations as dbop
from constants import TOKEN, STOCK_DAY, INDEX_DAY,TABLE,COLUMNS
import pandas as pd
db_type = "sqlite3"
conn = dbop.connect_db(db_type)
cursor = conn.cursor()

# Get all trading dates from stock index table.
cursor.execute("select * from {0}".format(
                    INDEX_DAY[TABLE]))
df_idx_day = pd.DataFrame(cursor.fetchall())
df_idx_day.columns = dbop.cols_from_cur(cursor)

# Get day data of all stocks from stocks table.
cursor.execute("select * from {0}".format(STOCK_DAY[TABLE]))
df_stocks_day = pd.DataFrame(cursor.fetchall())
df_stocks_day.columns = dbop.cols_from_cur(cursor)
print(df_stocks_day.shape, df_idx_day.shape)


(123701, 9) (17614, 7)


In [66]:
df_stck_na = df_stocks_day[df_stocks_day.isna().any(axis=1)]
df_idx_na = df_idx_day[df_idx_day.isna().any(axis=1)]

print("indexes:\t",df_idx_na)
# print("stocks",df_stck_na)
print("\nstocks:\t",df_stck_na.shape)
# # for code,df in df_stck_na.groupby("code"):
# #     print(df)

indexes:	 Empty DataFrame
Columns: [code, date, open, high, low, close, vol]
Index: []

stocks:	 (15314, 9)


In [67]:
dates = sorted(df_idx_day["date"].unique())
df_stocks_day = df_stocks_day.set_index("date")

In [69]:
print(len(dates), dates[0],dates[-1])
df_changed = pd.DataFrame(columns=df_stocks_day.iloc[[0,1]].reset_index().columns)
df_changed.set_index("date")
for code, df_single_stock in df_stocks_day.groupby(by="code"):
    df_single_stock = df_single_stock.sort_index()
    # start_date is the first date on which the whole row is not null in the
    #  table.
    # stock_dates are the dates on which the whole row is not null in the table.
    stock_dates = df_single_stock.index[df_single_stock.notnull().all(
        axis=1)]
    start_date = None
    na_dates = []
    for d in stock_dates:
        if d not in dates:
            na_dates.append(d)
        else:
            start_date = d
            break
    if not start_date:
        raise ValueError("start_date does not exist!")
    print(code, len(na_dates),na_dates[0] if na_dates else None,na_dates[-1] if na_dates else None)
    print("start:",start_date,"end:",dates[-1],"\n")
    
    start_idx = dates.index(start_date)
    
    for i in range(start_idx+1,len(dates)):
        is_changed = False
        row = None
        
        # Skip if the row exists and the whole row is not null.
        if dates[i] in df_single_stock.index and df_single_stock.loc[dates[i]].notnull().all().all():
            continue
        
        if dates[i] in df_single_stock.index:
            row = df_single_stock.loc[dates[i]].copy()
            # if df_single_stock.loc[dates[i]].notna().all().all():
            #     continue
            
            # Note that by using groupby code, code is always not null.
            # Fill prices open,high,low,close if all are null.
            if df_single_stock.loc[dates[i],["open","high","low","close"]].isna().all().all():
                is_changed = True
                df_single_stock.loc[dates[i],["open","high","low","close"]] = df_single_stock.loc[dates[i-1],["open","high","low","close"]]
                # Fill vol and amt if both are null. 
                # Do nothing if both are 0. 
                # Otherwise, raise error. 
                if df_single_stock.loc[dates[i],["vol","amt"]].isna().all().all():
                    df_single_stock.loc[dates[i],["vol","amt"]] = [0,0]
                elif not (df_single_stock.loc[dates[i],["vol","amt"]] == [0,0]).all().all():
                    raise ValueError("Error row: {}".format(df_single_stock.loc[dates[i]]))
                
                # Fill adj_factor if null.
                if df_single_stock.loc[dates[i],["adj_factor"]].isna().all().all():
                    df_single_stock.loc[dates[i],["adj_factor"]] = df_single_stock.loc[dates[i-1],["adj_factor"]]
            
            elif df_single_stock.loc[dates[i],["open","high","low","close"]].notnull().all().all():
                if not df_single_stock.loc[dates[i],["vol","amt"]].notnull().all().all():
                    raise ValueError("Error row: {}".format(df_single_stock.loc[dates[i]]))
                
                # Fill adj_factor if null.
                if df_single_stock.loc[dates[i],["adj_factor"]].isna().all().all():
                    is_changed = True
                    df_single_stock.loc[dates[i],["adj_factor"]] = df_single_stock.loc[dates[i-1],["adj_factor"]]
            else:
                raise ValueError("Error row: {}".format(df_single_stock.loc[dates[i]]))
         
        else:
            df_single_stock.loc[dates[i]] = df_single_stock.loc[dates[i-1]]
            df_single_stock.loc[dates[i],["vol","amt"]] = [0,0]
            is_changed = True
        
        if is_changed:
            df_changed.loc[dates[i]] = df_single_stock.loc[dates[i]]
            print("-"*10+"\n",dates[i], code)
            print(dict(df_single_stock.loc[dates[i-1]]))
            print(None if row is None else dict(row))
            print(dict(df_single_stock.loc[dates[i]]),"\n")

df_changed.reset_index()
print(df_changed)

4326 2001-01-02 2018-11-07
000001.SZ 0 None None
start: 2002-03-22 end: 2018-11-07 



000002.SZ 0 None None
start: 2002-03-22 end: 2018-11-07 



000063.SZ 0 None None
start: 2002-03-22 end: 2018-11-07 



000070.SZ 0 None None
start: 2002-03-22 end: 2018-11-07 

